In [1]:
print("Hello")

Hello


In [2]:
import pandas as pd
import os

In [3]:
business_file = '/kaggle/input/yelp-json/yelp_academic_dataset_business.json'
review_file = '/kaggle/input/yelp-json/yelp_academic_dataset_review.json'

print("Done")

Done


In [4]:
df_business = pd.read_json(business_file, lines=True)
print(df_business.head())

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.9555

# Choose what city state you want

In [5]:
CHOSEN_CITY = 'new orleans'
CHOSEN_STATE = 'la'

# Filter the businesses in that city state combo

In [6]:
df_business = pd.read_json(business_file, lines=True)

metropolitan_filter = (df_business['city'].str.lower() == CHOSEN_CITY) & (df_business['state'].str.lower() == CHOSEN_STATE)
df_filtered_business = df_business[metropolitan_filter].copy()

print("Done")

Done


In [7]:
filtered_business_file = '/kaggle/working/df_filtered_business.json'

df_filtered_business.to_json(
    filtered_business_file,
    orient='records',
    lines=True,
    index=False
)

print("Done")

Done


# Filter reviews for all the businesses in that city state combo

In [8]:
filtered_business_file = '/kaggle/working/df_filtered_business.json' 

CHUNK_SIZE = 50000 #capping the memory

df_filtered_business = pd.read_json(filtered_business_file, lines=True)
filtered_business_ids = df_filtered_business['business_id'].unique()

filtered_chunks = []
processed_chunks = 0
total_filtered_reviews = 0

print(f"\nStarting memory-safe filtering")

review_reader = pd.read_json(
    review_file,
    lines=True,
    chunksize=CHUNK_SIZE
)

for chunk in review_reader:
    review_filter = chunk['business_id'].isin(filtered_business_ids)
    filtered_chunk = chunk[review_filter]
    
    filtered_chunks.append(filtered_chunk)

    processed_chunks += 1
    total_filtered_reviews += len(filtered_chunk)
    
    if processed_chunks % 10 == 0:
        print(f"Processed {processed_chunks} chunks ({processed_chunks * CHUNK_SIZE} lines approx.). Filtered reviews: {total_filtered_reviews}")

df_filtered_review = pd.concat(filtered_chunks, ignore_index=True)

print("Done")


Starting memory-safe filtering
Processed 10 chunks (500000 lines approx.). Filtered reviews: 55456
Processed 20 chunks (1000000 lines approx.). Filtered reviews: 99400
Processed 30 chunks (1500000 lines approx.). Filtered reviews: 135989
Processed 40 chunks (2000000 lines approx.). Filtered reviews: 182237
Processed 50 chunks (2500000 lines approx.). Filtered reviews: 229739
Processed 60 chunks (3000000 lines approx.). Filtered reviews: 274332
Processed 70 chunks (3500000 lines approx.). Filtered reviews: 318109
Processed 80 chunks (4000000 lines approx.). Filtered reviews: 367230
Processed 90 chunks (4500000 lines approx.). Filtered reviews: 415553
Processed 100 chunks (5000000 lines approx.). Filtered reviews: 461619
Processed 110 chunks (5500000 lines approx.). Filtered reviews: 511628
Processed 120 chunks (6000000 lines approx.). Filtered reviews: 554581
Processed 130 chunks (6500000 lines approx.). Filtered reviews: 594388
Processed 140 chunks (7000000 lines approx.). Filtered re

In [9]:
filtered_review_file = '/kaggle/working/df_filtered_review.json'

df_filtered_review.to_json(filtered_review_file, orient='records', lines=True, index=False)

print("Done")

Done


# Verify

In [10]:
#Business Shape
df_filtered_business.shape

(6215, 14)

In [11]:
#Review Shape
df_filtered_review.shape

(635521, 9)